<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/10-colab-random-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# import utilities
import os
import json
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Fri Jul  3 16:20:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [8]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [9]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-07-03T16-30-04'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 117 (delta 2), reused 68 (delta 1), pack-reused 0
Receiving objects: 100% (117/117), 937.84 KiB | 9.67 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [10]:
os.chdir(os.environ['PROJECT_DIR'])

In [11]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-07-03T16-30-04
total 73
drwx------  4 root root  4096 Jul  3 16:30 data
drwx------  2 root root  4096 Jul  3 16:30 experiments
-rw-------  1 root root 35149 Jul  3 16:30 LICENSE
drwx------  2 root root  4096 Jul  3 16:30 notebooks
-rw-------  1 root root 10478 Jul  3 16:30 README.md
drwx------  3 root root  4096 Jul  3 16:30 reports
drwx------  2 root root  4096 Jul  3 16:30 requirements
drwx------ 13 root root  4096 Jul  3 16:30 src
drwx------  3 root root  4096 Jul  3 16:30 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-07-02T18-36-13/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
RANDOM_SEARCH_ID = 1
EXPERIMENT_ID = 0

In [17]:
def load_random_search_experiment_params(random_search_id, experiment_id):
  with open(f'../data/experiments/random_search_{random_search_id}.json', 'r') as f:
    experiments = json.load(f)
    loaded_params = experiments[f'experiment_{experiment_id}']
    id = f'0{experiment_id}' if experiment_id < 10 else experiment_id
    
    return {
      'dir_data': '../data/processed/subtoken/', # get the input data from the shared directory
      'file_checkpoint_dir': 'models/checkpoints/baseline/',
      'dir_preprocessed_data': 'data/processed/seq2seq/',

      'experiment_name': f'rs_{random_search_id}_{id}',
      'max_input_length': loaded_params['max_input_seq_length'],
      'max_output_length': loaded_params['max_output_seq_length'],
      'input_vocab_size': loaded_params['vocabulary_size'],
      'output_vocab_size': loaded_params['vocabulary_size'],
      'input_embedding_dim': loaded_params['embedding_dim'],
      'output_embedding_dim': loaded_params['embedding_dim'],
      'latent_dim': loaded_params['latent_dim'],
      'learning_rate': loaded_params['learning_rate'],
      'dropout_rate': loaded_params['dropout_rate'],
      'batch_size': loaded_params['batch_size'],

      'epochs': 200,
      'early_stopping_patience': 3,
      'early_stopping_min_delta': 0.0,
      'evaluation_dataset': 'validation',
      'random_seed': 1,
    }

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = load_random_search_experiment_params(
    random_search_id=1,
    experiment_id=0
)

print('params', params)

run(Namespace(**params))

Initializing logger
Current working directory: /content/gdrive/My Drive/src/identifier-suggestion-2020-07-02T18-36-13


wandb: Wandb version 0.9.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=512, dir_data='../data/processed/subtoken/', dir_preprocessed_data='data/processed/seq2seq/', dropout_rate=0.130622, early_stopping_min_delta=0.0, early_stopping_patience=4, epochs=200, evaluation_dataset='validation', experiment_name='random_search_0_10', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=96, input_vocab_size=6600, latent_dim=1024, learning_rate=0.046641, max_input_length=80, max_output_length=7, output_embedding_dim=96, output_vocab_size=6600, random_seed=1)
Preprocessed files not found. Preprocessing...
Reading input files


  7%|▋         | 45267/633714 [00:00<00:01, 452659.64it/s]

Adding <start> and <end> markers to output sequences


...

inputs after tokenizer 0    [7, 52, 303, 52, 9, 45, 154, 4, 11, 52, 3, 27,...
1    [7, 422, 4, 153, 217, 2604, 3, 852, 6, 221, 2,...
2    [7, 19, 3, 1296, 1872, 130, 126, 1872, 2, 7, 1...
Name: inputs, dtype: object
Encoding output sequences into numbers


100%|██████████| 78990/78990 [00:01<00:00, 51203.84it/s]


outputs after tokenizer 0          [2, 102, 73, 3]
1    [2, 32, 246, 2189, 3]
2             [2, 6227, 3]
Name: outputs, dtype: object
Padding and aligning input sequences
inputs after padding 0    [7, 52, 303, 52, 9, 45, 154, 4, 11, 52, 3, 27,...
1    [7, 422, 4, 153, 217, 2604, 3, 852, 6, 221, 2,...
2    [7, 19, 3, 1296, 1872, 130, 126, 1872, 2, 7, 1...
Name: inputs, dtype: object
Padding and aligning output sequences
Outputs after padding:  0       [2, 102, 73, 3, 0, 0, 0]
1    [2, 32, 246, 2189, 3, 0, 0]
2       [2, 6227, 3, 0, 0, 0, 0]
Name: outputs, dtype: object
Done preprocessing
Done preprocessing. Saving...


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['inputs', 'outputs'], dtype='object')]

  encoding=encoding,


Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  633600    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  4591616   
Total params: 5,225,216
Trainable params: 5,225,216
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
DecoderEmbedding (Embedding) multiple                  633600    
_________________________________________________________________
DecoderLSTM (LSTM)           multiple                  8785920   
________________________________________________________